In [ ]:
import json
import openai

openai.api_key = "YOUR_API_KEY"

In [11]:
#classifier by https://twitter.com/_ScottCondron/status/1670827747684364288
def classify(input_string: str) -> str:
    functions = [
        {
            "name": "print_sentiment",
            "description": "A function that prints the given sentiment",
            "parameters": {
                "type": "object",
                "properties": {
                    "sentiment": {
                        "type": "string",
                        "enum": ["positive", "negative", "neutral"],
                        "description": "The sentiment."
                    }
                },
                "required": ["sentiment"]
            }
        }
    ]

    messages = [
        {"role": "user", "content": input_string}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
       # log_level="info"
    )

    choices = response['choices']
    if len(choices) > 0 and 'message' in choices[0]:
        function_call = choices[0]['message']['function_call']
        argument = json.loads(function_call['arguments'])
        return argument

    return None



input_string = "I can't stand Twitter!"
result = classify(input_string)
print(result)

{'sentiment': 'negative'}


In [24]:
#added aspect categories, attempt at multiple sentiments per input does not work atm
def classify(input_string: str) -> list:
    functions = [
        {
            "name": "print_sentiment",
            "description": "Print all the given sentiments and aspect categories, may have multiple sentiments per input string",
            "parameters": {
                "type": "object",
                "properties": {
                    "sentiment_polarity": {
                        "type": "string",
                        "enum": ["positive", "negative", "neutral"],
                        "description": "The sentiment."
                    },
                    "aspect_category": {
                        "type": "string",
                        "enum": ["contender", "jury", "event"],
                        "description": "The aspect category."
                    }
                },
                "required": ["sentiment_polarity", "aspect_category"]
            }
        }
    ]

    messages = [
        {"role": "user", "content": input_string}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        # log_level="info"
    )

    results = []
    choices = response['choices']
    for choice in choices:
        if 'message' in choice:
            function_call = choice['message']['function_call']
            argument = json.loads(function_call['arguments'])
            results.append(argument)

    return results


input_string = "I can't stand this jury member, but I really like this contender!"
results = classify(input_string)
for result in results:
    print(result)


{'sentiment_polarity': 'negative', 'aspect_category': 'jury'}


In [23]:
#verbose version to see what it does
def classify(input_string: str) -> list:
    functions = [
        {
            "name": "print_sentiments",
            "description": "A function that prints all the given sentiment polarities and aspect categories",
            "parameters": {
                "type": "object",
                "properties": {
                    "sentiment_polarity": {
                        "type": "string",
                        "enum": ["positive", "negative", "neutral"],
                        "description": "The sentiment."
                    },
                    "aspect_category": {
                        "type": "string",
                        "enum": ["contender", "jury", "event"],
                        "description": "The aspect category."
                    }
                },
                "required": ["sentiment_polarity", "aspect_category"]
            }
        }
    ]

    messages = [
        {"role": "user", "content": input_string}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
 #       log_level="info"
    )

    results = []
    choices = response['choices']
    for choice in choices:
        if 'message' in choice:
            function_call = choice['message']['function_call']
            argument = json.loads(function_call['arguments'])
            results.append(argument)
    
    print(response)
    
    return results


input_string = "I can't stand this jury member, but I really like this contender!"
results = classify(input_string)
for result in results:
    print(result)


{
  "id": "chatcmpl-7TSlZ4CXgJtNfyvHHzlIFjeUxNANk",
  "object": "chat.completion",
  "created": 1687256445,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "print_sentiments",
          "arguments": "{\n  \"sentiment_polarity\": \"positive\",\n  \"aspect_category\": \"contender\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 105,
    "completion_tokens": 28,
    "total_tokens": 133
  }
}
{'sentiment_polarity': 'positive', 'aspect_category': 'contender'}


In [28]:
#via logits in MarvinAI https://twitter.com/AAAzzam/status/1669753722828730378
import tiktoken
import openai

openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role': 'system', 'content': 'You respond with whether the user input is happy, sad, or mad'},
        {'role': 'user', 'content': "I can't stand Twitter!"}
    ],
    logit_bias={
        '57621': 100,  # 57621 is the token for 'happy'
        '83214': 100,  # 83214 is the token for 'sad'
        '20920': 100   # 20920 is the token for 'mad'
    },
    max_tokens=1,  # You can only respond with a single token
    temperature=0
).choices[0].message.content


'mad'